<a href="https://colab.research.google.com/github/vijaysankar1985/transformers/blob/main/Transformers_ZTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers pandas numpy tqdm os

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [7]:
# Imports
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertModel
import numpy as np
import pandas as pd
from tqdm import tqdm
import argparse
import os
import requests

In [16]:
import kagglehub

path=kagglehub.dataset_download("uciml/news-aggregator-dataset")
print("Path to dataset files:", path)

# Download latest version
# path = kagglehub.dataset_download("uciml/news-aggregator-dataset")

# print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/news-aggregator-dataset


In [30]:
df=pd.read_csv(path+"/uci-news-aggregator.csv")
df.head(3)

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550


In [31]:
df_req=df[['TITLE','CATEGORY']]
df_req.head(3)

,TITLE,CATEGORY
0,"Fed official says weak data caused by weather,...",b
1,Fed's Charles Plosser sees high bar for change...,b
2,US open: Stocks fall after Fed official hints ...,b


In [32]:
df_req.CATEGORY.value_counts()

,count
CATEGORY,
e,152469
b,115967
t,108344
m,45639


In [33]:
df_req.shape

(422419, 2)

In [34]:
my_dict={
    'b':'Business',
    't':'Science',
    'e':'Entertainment',
    'm':'Health'
    }

In [35]:
def update_cat(x):
    return my_dict[x]

In [36]:
df_req['CATEGORY']=df_req['CATEGORY'].apply(lambda x: update_cat(x))

/tmp/ipython-input-3778833218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_req['CATEGORY']=df_req['CATEGORY'].apply(lambda x: update_cat(x))


In [38]:
df_req

,TITLE,CATEGORY
0,"Fed official says weak data caused by weather,...",Business
1,Fed's Charles Plosser sees high bar for change...,Business
2,US open: Stocks fall after Fed official hints ...,Business
3,"Fed risks falling 'behind the curve', Charles ...",Business
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,Business
...,...,...
422414,Surgeons to remove 4-year-old's rib to rebuild...,Health
422415,Boy to have surgery on esophagus after battery...,Health
422416,Child who swallowed battery to have reconstruc...,Health
422417,Phoenix boy undergoes surgery to repair throat...,Health


In [39]:
encode_dict={}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

In [40]:
df_req['ENCODE_CAT']=df_req['CATEGORY'].apply(lambda x: encode_cat(x))

/tmp/ipython-input-81967398.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_req['ENCODE_CAT']=df_req['CATEGORY'].apply(lambda x: encode_cat(x))


In [41]:
df_req

,TITLE,CATEGORY,ENCODE_CAT
0,"Fed official says weak data caused by weather,...",Business,0
1,Fed's Charles Plosser sees high bar for change...,Business,0
2,US open: Stocks fall after Fed official hints ...,Business,0
3,"Fed risks falling 'behind the curve', Charles ...",Business,0
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,Business,0
...,...,...,...
422414,Surgeons to remove 4-year-old's rib to rebuild...,Health,3
422415,Boy to have surgery on esophagus after battery...,Health,3
422416,Child who swallowed battery to have reconstruc...,Health,3
422417,Phoenix boy undergoes surgery to repair throat...,Health,3


In [42]:
encode_dict

{'Business': 0, 'Science': 1, 'Entertainment': 2, 'Health': 3}

In [43]:
encode_dict.keys()

dict_keys(['Business', 'Science', 'Entertainment', 'Health'])

In [44]:
encode_dict.values()

dict_values([0, 1, 2, 3])